In [1]:
import cudf, glob, gc, os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

VER = 93

In [2]:
files = glob.glob('../../data/train_data/*_parquet/*')

In [3]:
files[:4]

['/raid/Kaggle/otto/valid/test_parquet/004.parquet',
 '/raid/Kaggle/otto/valid/test_parquet/019.parquet',
 '/raid/Kaggle/otto/valid/test_parquet/008.parquet',
 '/raid/Kaggle/otto/valid/test_parquet/011.parquet']

In [4]:
%%time
for a,b in [(0,20),(20,40),(40,60),(60,80),(80,100),(100,120)]:
    print(f'Processing {b-a} files...')
    for k in range(a,b):
        df = cudf.read_parquet(files[k])
        df = df.sort_values(['session','ts'],ascending=[True,False])
        df = df.reset_index(drop=True)
        df['n'] = df.groupby('session').cumcount()
        df = df.loc[df.n<30].drop('n',axis=1)
        df = df.merge(df,on='session')
        df = df.loc[ ((df.ts_x - df.ts_y).abs()< 24 * 60 * 60) & (df.aid_x != df.aid_y) ]
        df = df[['session', 'aid_x', 'aid_y','ts_x','ts_y']].drop_duplicates(['session', 'aid_x', 'aid_y'])
        df['wgt'] = 1 + 3*(df.ts_x - 1659304800)/(1662328791-1659304800)
        df['wgt'] = df['wgt'] * ((1/2)**( (df.ts_x - df.ts_y).abs() /60/60))
        df = df[['aid_x','aid_y','wgt']]
        df.wgt = df.wgt.astype('float32')
        df = df.groupby(['aid_x','aid_y']).wgt.sum()
        if k==a: tmp2 = df
        else: tmp2 = tmp2.add(df, fill_value=0)
        print(k,', ',end='')
    print()
    if a==0: tmp = tmp2
    else: tmp = tmp.add(tmp2, fill_value=0)
    del tmp2, df
    gc.collect()

Processing 20 files...
0 , 1 , 2 , 3 , 4 , 5 , 6 , 7 , 8 , 9 , 10 , 11 , 12 , 13 , 14 , 15 , 16 , 17 , 18 , 19 , 
Processing 20 files...
20 , 21 , 22 , 23 , 24 , 25 , 26 , 27 , 28 , 29 , 30 , 31 , 32 , 33 , 34 , 35 , 36 , 37 , 38 , 39 , 
Processing 20 files...
40 , 41 , 42 , 43 , 44 , 45 , 46 , 47 , 48 , 49 , 50 , 51 , 52 , 53 , 54 , 55 , 56 , 57 , 58 , 59 , 
Processing 20 files...
60 , 61 , 62 , 63 , 64 , 65 , 66 , 67 , 68 , 69 , 70 , 71 , 72 , 73 , 74 , 75 , 76 , 77 , 78 , 79 , 
Processing 20 files...
80 , 81 , 82 , 83 , 84 , 85 , 86 , 87 , 88 , 89 , 90 , 91 , 92 , 93 , 94 , 95 , 96 , 97 , 98 , 99 , 
Processing 20 files...
100 , 101 , 102 , 103 , 104 , 105 , 106 , 107 , 108 , 109 , 110 , 111 , 112 , 113 , 114 , 115 , 116 , 117 , 118 , 119 , 
CPU times: user 55.5 s, sys: 41.1 s, total: 1min 36s
Wall time: 1min 40s


In [5]:
tmp = tmp.reset_index()

In [6]:
tmp = tmp.sort_values(['aid_x','wgt'],ascending=[True,False])
tmp = tmp.reset_index(drop=True)
tmp['n'] = tmp.groupby('aid_x').aid_y.cumcount()
tmp = tmp.loc[tmp.n<80].drop('n',axis=1)

In [7]:
print( tmp.shape )
tmp.head()

(95124522, 3)


,aid_x,aid_y,wgt
0,0,532042,13.643975
1,0,643097,12.432261
2,0,1848174,10.191021
3,0,1735605,7.988760
4,0,1363081,5.683196


In [8]:
%%time
df = tmp.to_pandas().groupby('aid_x').aid_y.apply(list)

CPU times: user 43 s, sys: 2.78 s, total: 45.8 s
Wall time: 45.8 s


In [10]:
%%time
import pickle
with open(f'../../data/covisit_matrices/top_80_aids_v{VER}.pkl', 'wb') as f:
    pickle.dump(df.to_dict(), f)

CPU times: user 3.71 s, sys: 676 ms, total: 4.39 s
Wall time: 4.49 s
